In [24]:
from dotenv import load_dotenv
from app.helpers import *
import os

load_dotenv()
coinmarketcap_api_key = os.getenv('COINMARKETCAP_API')

# if __name__ == "__main__":
    # print(get_map())
    # res = compute_holding(coinmarketcap_api_key, 'files/portfolio.json', 'EUR')

In [44]:
# from app.helpers import _load

api_key = os.getenv('COINMARKETCAP_API')
portfolio_filename = {'test': 'test'}
currency = 'EUR'

# def compute_holding(api_key, portfolio_filename, currency):
    """
    Compute the value of your portfolio given the quantity of each token owned

    :param api_key:            string,  the API key required to use the API
    :param portfolio_filename: string,  json file containing the info about your owned tokens
    :param currency:           string,  the currency wanted
    :return:                   dict,    value of each coins holded
    """
    try:
        portfolio = _load(portfolio_filename)
    except FileNotFoundError:
        sys.exit(f"No {portfolio_filename} file found")

    slugs = ','.join(portfolio['data'][el]['slug'] for el in portfolio['data'])

    headers = {'X-CMC_PRO_API_KEY': api_key}
    payload = {'slug': slugs, 'convert': currency}
    url = f"{COINMARKETCAP_API_URL}/cryptocurrency/quotes/latest"

    try:
        res = requests.get(url, headers=headers, params=payload)
    except requests.exceptions.RequestException as error:
        sys.exit(f"Something went wrong: {error}")
    if res.status_code != 200:
        sys.exit(f"Something went wrong with your call API: {res.status_code}\n{res.json()}")
    invested = portfolio['invested']['quantity']
    sum_, res, quotes = 0, {}, res.json()['data']
    for cur in quotes:
        hold = portfolio['data'][quotes[cur]['slug']]
        value = quotes[cur]['quote'][currency]['price'] * hold['quantity']
        sum_ += value
        res[quotes[cur]['symbol']] = {
            'price': quotes[cur]['quote'][currency]['price'],
            'quantity': hold['quantity'],
            'value': quotes[cur]['quote'][currency]['price'] * hold['quantity'],
            'currency': currency,
        }
    res['total'] = {
        'value': sum_,
        'currency': currency,
        'profit': sum_ - invested,
        'ROI': sum_ / invested
    }
    print res

{'ETH': {'price': 3979.9357942651945, 'quantity': 0.7722, 'value': 3073.306420331583, 'currency': 'EUR'}, 'ADA': {'price': 1.72600921715599, 'quantity': 712.181395, 'value': 1229.2316520570107, 'currency': 'EUR'}, 'FLUX': {'price': 1.140278520849111, 'quantity': 1226.22438311, 'value': 1398.2373258017844, 'currency': 'EUR'}, 'SOL': {'price': 219.02430142725967, 'quantity': 10.79697, 'value': 2364.79881178108, 'currency': 'EUR'}, 'AVAX': {'price': 72.06477750481199, 'quantity': 49.710926, 'value': 3582.406821748173, 'currency': 'EUR'}, 'ALEPH': {'price': 0.4239450219456522, 'quantity': 762.6479, 'value': 323.32078070230557, 'currency': 'EUR'}, 'SRM': {'price': 6.696832441101963, 'quantity': 42.43859, 'value': 284.20412626662534, 'currency': 'EUR'}, 'BETH': {'price': 3688.8031297497446, 'quantity': 0.97629, 'value': 3601.341607543378, 'currency': 'EUR'}, 'total': {'value': 15856.847546231942, 'currency': 'EUR', 'profit': 4856.847546231942, 'ROI': 1.4415315951119947}}


In [29]:
coinmarketcap_api_key = os.getenv('COINMARKETCAP_API')
compute_holding(coinmarketcap_api_key, 'files/portfolio.json', 'EUR')

KeyError: 'ETH'

In [41]:
# portfolio
portfolio['data'][quotes['1027']['slug']]

{'slug': 'ethereum', 'quantity': 0.7722, 'where': 'Metamask'}